# scratchpad


## Step 1: Define the Agent's State
This is the central "memory" of our agent. All information the agent needs to operate will be stored here. In LangGraph, the state is mutable, meaning it can be modified by any node in the graph, and it's passed from one node to the next.

Why we use it: The state allows us to build a stateful system instead of a simple, stateless chain of events. A stateless system would have to re-evaluate the entire problem from scratch at every step. By passing a state object, we can maintain context and accumulate information over multiple steps, which is crucial for a complex task like research.

In [ ]:
from typing import TypedDict, List

class AgentState(TypedDict):
    """
    Represents the state of our multi-step research agent.
    This is basically a schema

    Attributes:
        query: The user's original research query.
        research_plan: A list of search queries or sub-tasks to execute.
        research_results: A list of raw results gathered from the tools.
        report: The final, synthesized report.
    """
    query: str
    research_plan: List[str]
    research_results: List[str]
    report: str



## Step 3: Implement the Tools
Tools are the "hands" of our agent. They are functions that the agent's LLM can call to interact with external systems, such as search engines, databases, or other APIs.

Why we use it: Agents, by themselves, are limited to the knowledge they were trained on. By giving them access to tools, we enable them to overcome this limitation, allowing them to perform real-time searches, access up-to-date information, and perform complex calculations.

In [10]:
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
print(f"tool name: {search.name}")
print(f"tool description: {search.description}")
print(f"tool arg schema: {search.args_schema}")


from langchain_community.tools.ddg_search.tool import DDGInput
print(DDGInput.model_fields)
print("#"*40)

query = "Who is the prime minister of Italy?"
search_input = DDGInput(query=query)
result = search.invoke(search_input.dict())
print(result)

tool name: duckduckgo_search
tool description: A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
tool arg schema: <class 'langchain_community.tools.ddg_search.tool.DDGInput'>
{'query': FieldInfo(annotation=str, required=True, description='search query to look up')}
########################################


/tmp/ipykernel_695944/1189855989.py:14: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  result = search.invoke(search_input.dict())


The prime minister of Italy , officially the president of the Council of Ministers , is the head of government of the Italian Republic. The office of president of the Council of Ministers is established by articles 92–96 of the Constitution of Italy ... Giorgia Meloni is an Italian politician who has served as Prime Minister of Italy since 2022. She is the first woman to hold the office. A member of the Chamber of Deputies since 2006, she has been president of the right-wing to far-right Brothers of ... Prime Minister Giorgia Meloni and European MP Alessandra Moretti were among the high-profile Italian women whose images were subject to lewd and violent content. Prime Minister Giorgia Meloni during the Rimini Meeting 2025 in Rimini, northeast Italy on August 27, 2025. Italian Prime Minister Georgia Meloni, pictured in March, has voiced outrage over a porn site that allegedly featured doctored images of herself and other high-profile women. ITALIAN PRIME MINISTER Giorgia Meloni has paid

# Deep research !

In [11]:
import os
from dotenv import load_dotenv

load_dotenv()

# Optional: Configure LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Deep Research Notebook"

print("Environment is set up and keys are loaded.")
print(os.getenv("TAVILY_API_KEY"))

Environment is set up and keys are loaded.
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


## we need to define tools and state of the agents

### TOOLS

In [ ]:
from typing import TypedDict, List, Optional # for state types
from typing_extensions import Annotated # for reducers
from langgraph.graph import add_messages #reducer
import operator
from langchain.tools import Tool #base class that will use to wrap the utilies
from langchain_community.tools.tavily_search import TavilySearchResults #already a tool
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_core.tools import StructuredTool
from langchain_core.messages import ToolMessage
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableConfig
import json
from langchain_openai import ChatOpenAI



#already subclass BaseModel and provide args_schema
tavily_tool = TavilySearchResults(max_results = 5)
print(f"tavily tool schema: {json.dumps(tavily_tool.args_schema.model_json_schema(), indent=2)}",end="\n\n")
print("#"*30)

#--------CUSTOM TOOLS-----------#
#define schema for the wrapper tools.
class ArxivSearchInput(BaseModel):
    query: str = Field(description="The search query for scientific papers on ArXiv.") #since i have not provided default, it treats it as required

# define the function WITH a config parameter
def arxiv_search_fn(query: str, config: RunnableConfig) -> str:
    # Read runtime knobs that you DON'T want the LLM to fill
    cfg = (config or {}).get("configurable", {})
    top_k: int = int(cfg.get("arxiv_top_k", 3))
    date_from: Optional[str] = cfg.get("arxiv_date_from")  # example extra knob

    wrapper = ArxivAPIWrapper(top_k_results=top_k, date_from=date_from)
    
    return wrapper.run(query)

arxiv_tool = StructuredTool.from_function(
    name="arxiv_search",
    description="Search ArXiv for scientific papers.",
    args_schema=ArxivSearchInput,
    func=arxiv_search_fn
)

print(f"arxiv tool schema: {json.dumps(arxiv_tool.args_schema.model_json_schema(),indent=2)}",end="\n\n")

class WikipediaSearchInput(BaseModel):
    query: str = Field(description="The search query for general knowledge on Wikipedia.")


def wikipedia_search_fn(query: str, config: RunnableConfig) -> str:
    cfg = (config or {}).get("configurable", {})
    top_k: int = int(cfg.get("wikipedia_top_k", 3))
    lang: Optional[str] = cfg.get("wikipedia_lang","en")
    wrapper = WikipediaAPIWrapper(top_k_results=top_k,lang=lang)

    return wrapper.run(query)

wikipedia_tool = StructuredTool.from_function(
    name="wikipedia_search",
    description="Search Wikipedia for general knowledge.",
    args_schema=WikipediaSearchInput,
    func=wikipedia_search_fn
)

print(f"wikipedia tool schema: {json.dumps(wikipedia_tool.args_schema.model_json_schema(),indent=2)}",end="\n\n")


research_tools = [tavily_tool, arxiv_tool, wikipedia_tool]
print(f"Tools created: {[tool.name for tool in research_tools]}")




tavily tool schema: {
  "description": "Input for the Tavily tool.",
  "properties": {
    "query": {
      "description": "search query to look up",
      "title": "Query",
      "type": "string"
    }
  },
  "required": [
    "query"
  ],
  "title": "TavilyInput",
  "type": "object"
}

##############################
arxiv tool schema: {
  "properties": {
    "query": {
      "description": "The search query for scientific papers on ArXiv.",
      "title": "Query",
      "type": "string"
    }
  },
  "required": [
    "query"
  ],
  "title": "ArxivSearchInput",
  "type": "object"
}

wikipedia tool schema: {
  "properties": {
    "query": {
      "description": "The search query for general knowledge on Wikipedia.",
      "title": "Query",
      "type": "string"
    }
  },
  "required": [
    "query"
  ],
  "title": "WikipediaSearchInput",
  "type": "object"
}

Tools created: ['tavily_search_results_json', 'arxiv_search', 'wikipedia_search']


In [64]:
# ------ TAVILY -------
#invocation test
tavily_result = tavily_tool.invoke({"query": "Is Israel currently at war?"}) #invoke expects dict matching schema
print(len(tavily_result)) #list of dicts
print(tavily_result[0].keys())
print(tavily_result[0]["content"][:200],end="\n\n\n")


# ---- WIKPEDIA -----

#custom tooling can be either used in this way:
llm = llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_completion_tokens=256)
llm_with_tools = llm.bind_tools([wikipedia_tool])
wiki_res = llm_with_tools.invoke("Who is the Prime Minister of Italy?")
print(wiki_res)
print(f"wiki_res TOOL CALL: {wiki_res.tool_calls}")
tool_calls = wiki_res.tool_calls
tool_outputs = []
for tool_call in tool_calls:
    print(f"\n--- Executing Tool: {tool_call['name']} ---")
    
    # Get the result from your actual tool function
    tool_output = wikipedia_tool.invoke(tool_call['args'])
    
    # Package the result in a ToolMessage with the correct ID
    tool_outputs.append(
        ToolMessage(content=str(tool_output), tool_call_id=tool_call["id"])
    )

print(f"\n--- Tool Output (first 200 chars): ---\n{tool_outputs[0].content[:200]}...")

#or this way
wikipedia_result = wikipedia_tool.invoke("Chi è il primo ministro italiano?",config={"configurable": {"wikipedia_top_k": 5, "wikipedia_lang": "it"}}
)
print(wikipedia_result[:200],end="\n\n\n")


#----- ARXIV ------

arxiv_res = arxiv_tool.invoke("What is Chain of Thought?")
print(arxiv_res[:400])

5
dict_keys(['title', 'url', 'content', 'score'])
Follow the latest news on the Israel Gaza war. Get live updates, maps, verified video reports and expert analysis from BBC journalists on the ground.


content='' additional_kwargs={'tool_calls': [{'id': 'call_vde9wqnqd7v4bDRTfpRIkKdp', 'function': {'arguments': '{"query":"Prime Minister of Italy"}', 'name': 'wikipedia_search'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 61, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CB3Szo2m8JBh0Kg96kMlAhh89Gyty', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--66fdda58-0422-4f45-8ccf-36506360655a-0' tool_calls=[{'name': 

### AGENTS

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


#define structure of planner output
class ResearchPlan(BaseModel):
    main_query: str = Field(description="A concise, focused query for search engines based on the initial task.")
    steps: List[str] = Field(description="A list of detailed, sequential steps to execute for the research.")



planner_llm = ChatOpenAI(model="gpt-5-nano",temperature=0,)
planner_prompt = ChatPromptTemplate.from_template(
    """You are an expert research director. Your goal is to create a detailed, step-by-step research plan 
    to answer the user's task.

    Task: {task}
    
    Generate a research plan with a main search query and a list of steps.""")

print(planner_prompt)

planner_chain = planner_prompt | planner_llm.with_structured_output(schema=ResearchPlan)

print("--- Testing the Planner Agent ---")
test_plan = planner_chain.invoke({"task": "What is the future of Retrieval-Augmented Generation?"})
print(f"Generated Main Query: {test_plan.main_query}")
print("Generated Steps:")
for i, step in enumerate(test_plan.steps, 1):
    print(f"{i}. {step}")

input_variables=['task'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['task'], input_types={}, partial_variables={}, template="You are an expert research director. Your goal is to create a detailed, step-by-step research plan \n    to answer the user's task.\n\n    Task: {task}\n\n    Generate a research plan with a main search query and a list of steps."), additional_kwargs={})]
--- Testing the Planner Agent ---
Generated Main Query: What is the future trajectory and key drivers of Retrieval-Augmented Generation (RAG) in AI systems, including architectures, data strategies, evaluation metrics, and deployment considerations.
Generated Steps:
1. 1) Define scope, success criteria, and time horizons (short-term: 1-2 years; mid-term: 3-5 years; long-term: 5-10+ years) for the RAG future study; specify target domains (open-domain QA, code, scientific literature, multi-modal), and outcomes (theory, benchmarks, roadmap).
2. 2)

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain import hub

executor_llm = ChatOpenAI(model="gpt-5-nano",temperature=0,)

#careful if using custom prompt, might mess up stuff
execution_agent = create_react_agent(model=executor_llm,tools= research_tools,)#prompt=execution_prompt)

print("\n--- Testing the Execution Agent ---")
# Let's test it on a single, simple task.
test_step = "Search for the latest developments in multi-modal RAG systems."
# The agent expects its input in a specific format: a dictionary with a 'messages' key.
executor_output = execution_agent.invoke({"messages": [("user", test_step)]})

# The agent's final answer is always in the last message of the output.
final_answer = executor_output['messages'][-1].content
print(f"\nExecutor's response to the test step:\n{final_answer}")

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'} template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

--- Testing the Execution Agent ---

Executor's response to the test step:
Here’s a concise, up-to-date overview of the latest develo

In [ ]:
# if you want to inspect, use stream. 
query = "Latest developments in multimodal RAG systems."

for event in execution_agent.stream({"messages": [("user", query)]}):
    # Each event is a *delta* keyed by node name, e.g. "agent" or "tools"
    update = event.get("agent", event.get("tools", {}))
    for msg in update.get("messages", []):
        msg.pretty_print()        # nice formatting of AI/Human/Tool messages


================================== Ai Message ==================================
Tool Calls:
  arxiv_search (call_apHMzcBCznQoD06E9nXM3vg9)
 Call ID: call_apHMzcBCznQoD06E9nXM3vg9
  Args:
    query: multimodal retrieval augmented generation
  arxiv_search (call_qG4tcaCnizJIpmi1sCjluQGL)
 Call ID: call_qG4tcaCnizJIpmi1sCjluQGL
  Args:
    query: multimodal RAG 2023 2024 2025
  tavily_search_results_json (call_7p98qmn914B7pRaEbsr4vtKT)
 Call ID: call_7p98qmn914B7pRaEbsr4vtKT
  Args:
    query: multimodal RAG latest developments 2024 2025
  wikipedia_search (call_36iCysIIuTQ8JePOx0jYcUzL)
 Call ID: call_36iCysIIuTQ8JePOx0jYcUzL
  Args:
    query: Retrieval-augmented generation multimodal
================================= Tool Message =================================
Name: arxiv_search

Published: 2025-03-03
Title: MMed-RAG: Versatile Multimodal RAG System for Medical Vision Language Models
Authors: Peng Xia, Kangyu Zhu, Haoran Li, Tianze Wang, Weijia Shi, Sheng Wang, Linjun Zhang, James 